Importing libraries

In [1]:
from bs4 import BeautifulSoup
import requests

Getting the table from the website

In [2]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source,'lxml')

Getting the headings of the table

In [3]:
table = soup.find('table',class_ = 'wikitable sortable')
head = []
for heading in table.find_all('th'):
    head.append(heading.text)
head[2] = head[2].replace('\n','')
head[1] = head[1].replace('\n','')
print(head)

['Postcode', 'Borough', 'Neighbourhood']


Crearing a dataframe

In [4]:
import pandas as pd
df = pd.DataFrame(columns=head)
print(df)

Empty DataFrame
Columns: [Postcode, Borough, Neighbourhood]
Index: []


Inserting values

In [5]:
for row in table.find_all('tr'):
    temp=[]
    for data in row.find_all('td'):
        temp.append(data.text)
    if len(temp) > 0:
        df = df.append({'Postcode':temp[0],'Borough':temp[1],'Neighbourhood':temp[2]},ignore_index=True)

In [6]:
print(df.head())

  Postcode           Borough       Neighbourhood
0      M1A      Not assigned      Not assigned\n
1      M2A      Not assigned      Not assigned\n
2      M3A        North York         Parkwoods\n
3      M4A        North York  Victoria Village\n
4      M5A  Downtown Toronto      Harbourfront\n


Cleaning the data

In [7]:
n = df['Neighbourhood'].values
new=[]
for i in n:
    #print(i)
    i = i.replace('\n','')
    new.append(i)
#print(new)
df['Neighbourhood'] = new
#print(df.head())

n = df['Borough'].values
new=[]
for i in n:
    #print(i)
    i = i.replace('\n','')
    new.append(i)
#print(new)
df['Borough'] = new
print(df.head())

  Postcode           Borough     Neighbourhood
0      M1A      Not assigned      Not assigned
1      M2A      Not assigned      Not assigned
2      M3A        North York         Parkwoods
3      M4A        North York  Victoria Village
4      M5A  Downtown Toronto      Harbourfront


Removing rows where Borough is not assigned

In [8]:
df = df[df['Borough']!='Not assigned']
print(df.head())

  Postcode           Borough     Neighbourhood
2      M3A        North York         Parkwoods
3      M4A        North York  Victoria Village
4      M5A  Downtown Toronto      Harbourfront
5      M5A  Downtown Toronto       Regent Park
6      M6A        North York  Lawrence Heights


Grouping rows by Postcode

In [9]:
foo = lambda a: ','.join(a)
df = df.groupby('Postcode',as_index=False).agg({'Borough':'first','Neighbourhood':foo}).reindex(columns=df.columns)
print(df.head())

  Postcode      Borough                         Neighbourhood
0      M1B  Scarborough                         Rouge,Malvern
1      M1C  Scarborough  Highland Creek,Rouge Hill,Port Union
2      M1E  Scarborough       Guildwood,Morningside,West Hill
3      M1G  Scarborough                                Woburn
4      M1H  Scarborough                             Cedarbrae


In [10]:
print(df.shape)

(103, 3)
